In [12]:
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import os
import torch
import torch.utils.data as data 

from copy import deepcopy
from omegaconf import OmegaConf
from torchvision import transforms
from torch.nn.parallel import DistributedDataParallel as DDP
# 
from contrastive_learning.tests.test_model import load_lin_model, predict_traj_actions
from contrastive_learning.tests.animate_markers import AnimateMarkers
from contrastive_learning.tests.animate_rvec_tvec import AnimateRvecTvec
from contrastive_learning.datasets.dataloaders import get_dataloaders

from contrastive_learning.models.custom_models import LinearInverse
from contrastive_learning.datasets.state_dataset import StateDataset
from contrastive_learning.tests.plotting import plot_rvec_tvec, plot_corners
from contrastive_learning.datasets.dataloaders import get_dataloaders

### Model Loading
Create the distributed group
Load the linear inverse model from the saved path

In [2]:
# Start the multiprocessing to load the saved models properly
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "29505"

torch.distributed.init_process_group(backend='gloo', rank=0, world_size=1)
torch.cuda.set_device(0)

In [3]:
# Set the device and out_dir
device = torch.device('cuda:0')
out_dir = '/home/irmak/Workspace/DAWGE/contrastive_learning/out/2022.08.01/20-14_pli_ref_dog_lf_mse_fi_1_pt_corners_bs_64_hd_64_lr_0.001_zd_8'
cfg = OmegaConf.load(os.path.join(out_dir, '.hydra/config.yaml'))
if not ('pos_ref' in cfg):
    cfg.pos_ref = 'global'
model_path = os.path.join(out_dir, 'models/lin_model.pt')

# Load the encoder
lin_model = load_lin_model(cfg, device, model_path)

In [4]:
print(lin_model)

DistributedDataParallel(
  (module): LinearInverse(
    (model): Sequential(
      (0): Linear(in_features=32, out_features=64, bias=True)
      (1): ReLU()
      (2): Linear(in_features=64, out_features=16, bias=True)
      (3): ReLU()
      (4): Linear(in_features=16, out_features=2, bias=True)
    )
  )
)


In [5]:
print(cfg)

{'agent': {'_target_': 'contrastive_learning.models.agents.pli.PLI', 'loss_fn': 'mse', 'use_encoder': False, 'model': '???', 'optimizer': '???'}, 'optimizer': {'_target_': 'torch.optim.Adam', 'params': '???', 'lr': '???', 'weight_decay': '???'}, 'model': {'_target_': 'contrastive_learning.models.custom_models.LinearInverse', 'input_dim': '???', 'action_dim': '???', 'hidden_dim': '???'}, 'pos_encoder': {'_target_': 'contrastive_learning.models.custom_models.PosToEmbedding', 'input_dim': '???', 'hidden_dim': '???', 'out_dim': '???'}, 'seed': 42, 'device': 'cuda', 'agent_type': 'pli', 'dataset_type': 'state', 'pos_type': 'corners', 'pos_ref': 'dog', 'train_epochs': 1000, 'save_frequency': 10, 'train_dset_split': 0.8, 'batch_size': 64, 'lr': 0.001, 'weight_decay': 1e-05, 'z_dim': 8, 'pos_dim': 8, 'hidden_dim': 64, 'action_dim': 2, 'distributed': True, 'num_workers': 4, 'world_size': 1, 'num_gpus': 4, 'fps': 15, 'frame_interval': 1, 'video_type': 'color', 'experiment': '${agent_type}_ref_${

### Action Animation
1. Dump every predicted action for given data directory
2. Save the predicted and current action in a video

In [6]:
demo_name = 'box_marker_test_6'
data_dir = '/home/irmak/Workspace/DAWGE/src/dawge_planner/data/box_orientation_2_demos/test_demos/{}'.format(demo_name)
dump_dir = '/home/irmak/Workspace/DAWGE/contrastive_learning/tests/animations'
exp_name = '{}_{}'.format(out_dir.split('/')[-2], out_dir.split('/')[-1])
dump_file = '{}_{}.mp4'.format(demo_name, exp_name)

fps = 15

In [7]:
print('dump_file: {}'.format(dump_file))

dump_file: box_marker_test_6_2022.08.01_20-14_pli_ref_dog_lf_mse_fi_1_pt_corners_bs_64_hd_64_lr_0.001_zd_8.mp4


In [8]:
def get_curr_state(cfg, data_loader_iter):
    batch = next(data_loader_iter)
    curr_pos, next_pos, action = [b for b in batch]
    action = all_dset.denormalize_action(action[0].detach().numpy())
    if cfg.pos_type == 'corners':
        curr_pos, next_pos = all_dset.denormalize_corner(curr_pos[0].detach().numpy()), all_dset.denormalize_corner(next_pos[0].detach().numpy())
    elif cfg.pos_type == 'rvec_tvec':
        curr_pos, next_pos = all_dset.denormalize_pos_rvec_tvec(curr_pos[0].detach().numpy()), all_dset.denormalize_pos_rvec_tvec(next_pos[0].detach().numpy())
        
    return curr_pos, next_pos, action

In [9]:
def trans_pos_to_model(cfg, curr_pos): # Returns flattened positions
    if cfg.pos_type == 'corners':
        curr_pos = torch.FloatTensor(all_dset.normalize_corner(curr_pos))
        return torch.flatten(curr_pos)
    elif cfg.pos_type == 'rvec_tvec':
        curr_pos = torch.FloatTensor(all_dset.normalize_rvec_tvec(curr_pos))
        return curr_pos

In [10]:
def get_reference(cfg, curr_pos): # Gets flattened positions, shape: (16)
    ref_tensor = torch.zeros((curr_pos.shape))
    half_idx = int(curr_pos.shape[0] / 2) # In order not to have a control for pos_type
    if cfg.pos_ref == 'dog':
        ref_tensor = curr_pos[half_idx:]
        ref_tensor = ref_tensor.repeat(1,2)
    elif cfg.pos_ref == 'box':
        ref_tensor = curr_pos[:half_idx]
        ref_tensor = ref_tensor.repeat(1,2)
    return ref_tensor

In [13]:
# Get the dataset
global_cfg = deepcopy(cfg)
global_cfg.pos_ref = 'global'
dataset = StateDataset(global_cfg, single_dir=True, single_dir_root=data_dir)
test_loader = data.DataLoader(dataset, batch_size=1, shuffle=False, num_workers=4)
test_loader_iter = iter(test_loader)
_, _, all_dset = get_dataloaders(global_cfg)

DATASET POS_REF: global
len(dataset): 64
self.action_min: [ 0.         -0.30000001], self.action_max: [0.15000001 0.30000001]
DATASET POS_REF: global
len(dataset): 2683
self.action_min: [-0.15000001 -0.30000001], self.action_max: [0.15000001 0.30000001]


In [14]:
N = len(dataset)

curr_traj = np.zeros((N, cfg.pos_dim,2))
next_traj = np.zeros((N, cfg.pos_dim,2))
real_actions = np.zeros((N, 2))
pred_actions = np.zeros((N, 2))

for i in range(N):
    curr_pos, next_pos, action = get_curr_state(cfg, test_loader_iter)
    real_actions[i,:] = action
    curr_traj[i,:] = curr_pos
    next_traj[i,:] = next_pos
    
    # Normalize the current and next pos before inputting to the linear model
    curr_pos, next_pos = trans_pos_to_model(cfg, curr_pos).to(device), trans_pos_to_model(cfg, next_pos).to(device)
    ref_tensor = get_reference(cfg, curr_pos)
    
    pred_action = lin_model(curr_pos-ref_tensor, next_pos-ref_tensor)
    pred_action = all_dset.denormalize_action(pred_action[0].cpu().detach().numpy())
    
    pred_actions[i,:] = pred_action

In [15]:
nrows = 4
ncols = math.ceil(N / nrows)
print(N, ncols)
fig, axs = plt.subplots(figsize=(ncols*10,nrows*10), nrows=nrows, ncols=ncols) # Draw the predicted action
print(axs.shape)
fig.suptitle("Frame-by-frame Predictions")

for i in range(N):
    axs_row = int(i / ncols)
    axs_col = int(i % ncols)

    real_action = real_actions[i]
    pred_action = pred_actions[i]
    
    curr_pos = curr_traj[i]
    next_pos = next_traj[i]
    
    axs[axs_row,axs_col].set_title("Frame: {}".format(i))
    if cfg.pos_type == 'corners':
        _, frame_axis = plot_corners(axs[axs_row,axs_col], curr_pos, plot_action=True, actions=[real_action, pred_action], color_scheme=1)
        plot_corners(axs[axs_row,axs_col], next_pos, plot_action=False, use_frame_axis=True, frame_axis=frame_axis, color_scheme=2)
    elif cfg.pos_type == 'rvec_tvec':
        _, frame_axis = plot_rvec_tvec(axs[axs_row,axs_col], curr_pos, plot_action=True, actions=[real_action], color_scheme=1)
        plot_rvec_tvec(axs[axs_row,axs_col], next_pos, plot_action=True, actions=[pred_action], use_frame_axis=True, frame_axis=frame_axis, color_scheme=2)

plt_file_name = '{}_{}.png'.format(demo_name, exp_name)
plt.savefig(os.path.join('/home/irmak/Workspace/DAWGE/contrastive_learning/tests/plots', plt_file_name))

64 16
(4, 16)


In [49]:
# Get the dataset
# global_cfg = deepcopy(cfg)
# global_cfg.pos_ref = 'global'
# dataset = StateDataset(global_cfg, single_dir=True, single_dir_root=data_dir)
# predicted_actions = np.zeros((len(dataset), 2))
# test_loader = data.DataLoader(dataset, batch_size=cfg.batch_size, shuffle=False, num_workers=4)

# for i, batch in enumerate(test_loader):
#     curr_pos, next_pos, action = [b.to(device) for b in batch]
    
#     # Normalize the current and next pos before inputting to the linear model
#     ref_tensor = torch.zeros((curr_pos.shape))
#     half_idx = int(curr_pos.shape[1] / 2) # In order not to have a control for pos_type
#     if cfg.pos_ref == 'dog':
#         ref_tensor = curr_pos[:,half_idx:]
#         ref_tensor = ref_tensor.repeat(1,2)
#     elif cfg.pos_ref == 'box':
#         ref_tensor = curr_pos[:,:half_idx]
#         ref_tensor = ref_tensor.repeat(1,2)
    
#     pred_action = lin_model(curr_pos-ref_tensor, next_pos-ref_tensor)

#     print('Actual Action \t Predicted Action')
#     for j in range(len(action)):
#         print('{}, \t{}'.format(np.around(dataset.denormalize_action(action[j][0].cpu().detach().numpy()), 2),
#                                           dataset.denormalize_action(pred_action[j][0].cpu().detach().numpy())))
#         predicted_actions[i*cfg.batch_size+j,:] = dataset.denormalize_action(pred_action[j][0].cpu().detach().numpy())

# with open(os.path.join(data_dir, 'predicted_actions.npy'), 'wb') as f:
#     np.save(f, predicted_actions)

In [50]:

# if cfg.pos_type == 'corners':
#     AnimateMarkers(
#         data_dir = data_dir, 
#         dump_dir = dump_dir, 
#         dump_file = dump_file, 
#         fps = fps,
#         mult_traj = False,
#         show_predicted_action = True 
#     ) # Saves them in the given dump_file
# elif cfg.pos_type == 'rvec_tvec':
#     AnimateRvecTvec(
#         data_dir = data_dir, 
#         dump_dir = dump_dir, 
#         dump_file = dump_file,
#         fps = fps,
#         show_predicted_action=True
#     )

### Action Prediction Image
Predict the action for each frame in the test dataset and dump them in a grid image

In [17]:
from copy import deepcopy
cfg.batch_size = 32
global_cfg = deepcopy(cfg)
global_cfg.pos_ref = 'global'
train_loader, test_loader, dataset = get_dataloaders(global_cfg)

DATASET POS_REF: global
len(dataset): 2613
self.action_min: [-0.15000001 -0.30000001], self.action_max: [0.15000001 0.30000001]


In [18]:
len(test_loader.dataset)
batch = next(iter(test_loader))
curr_pos, next_pos, action = [b.to(device) for b in batch]
print('curr_pos: {}, next_pos: {}'.format(curr_pos, next_pos))

# Normalize the current and next pos before inputting to the linear model
ref_tensor = torch.zeros((curr_pos.shape))
half_idx = int(curr_pos.shape[1] / 2) # In order not to have a control for pos_type
if cfg.pos_ref == 'dog':
    ref_tensor = curr_pos[:,half_idx:]
    ref_tensor = ref_tensor.repeat(1,2)
elif cfg.pos_ref == 'box':
    ref_tensor = curr_pos[:,:half_idx]
    ref_tensor = ref_tensor.repeat(1,2)

pred_action = lin_model(curr_pos-ref_tensor, next_pos-ref_tensor)

curr_pos.shape

curr_pos: tensor([[0.5562, 0.5154, 0.5277, 0.4888, 0.5472, 0.4580, 0.5757, 0.4832, 0.6181,
         0.5098, 0.6474, 0.4972, 0.6653, 0.5322, 0.6352, 0.5462],
        [0.1600, 0.5091, 0.1592, 0.4692, 0.1967, 0.4573, 0.1987, 0.4972, 0.1287,
         0.6022, 0.1686, 0.5896, 0.1686, 0.6359, 0.1254, 0.6471],
        [0.4691, 0.1653, 0.4910, 0.1569, 0.5041, 0.1793, 0.4813, 0.1877, 0.4381,
         0.1877, 0.4503, 0.2073, 0.4267, 0.2157, 0.4145, 0.1947],
        [0.2769, 0.1751, 0.2557, 0.1975, 0.2362, 0.1807, 0.2573, 0.1597, 0.2834,
         0.1555, 0.2638, 0.1429, 0.2818, 0.1218, 0.3013, 0.1345],
        [0.6832, 0.1036, 0.7068, 0.1064, 0.7109, 0.1275, 0.6865, 0.1232, 0.6539,
         0.1120, 0.6564, 0.1331, 0.6327, 0.1275, 0.6311, 0.1078],
        [0.3029, 0.1485, 0.2761, 0.1597, 0.2687, 0.1373, 0.2948, 0.1289, 0.3156,
         0.1127, 0.3217, 0.0931, 0.3457, 0.0945, 0.3396, 0.1134],
        [0.1702, 0.5196, 0.1653, 0.4790, 0.2036, 0.4636, 0.2093, 0.5028, 0.0651,
         0.8067, 0.0936, 0.

torch.Size([32, 16])

In [19]:
print(curr_pos[0])

tensor([0.5562, 0.5154, 0.5277, 0.4888, 0.5472, 0.4580, 0.5757, 0.4832, 0.6181,
        0.5098, 0.6474, 0.4972, 0.6653, 0.5322, 0.6352, 0.5462],
       device='cuda:0')


In [20]:
nrows = 4
ncols = int(cfg.batch_size / nrows)
fig, axs = plt.subplots(figsize=(ncols*10,nrows*10), nrows=nrows, ncols=ncols) # Draw the predicted action
print(axs.shape)

for i in range(cfg.batch_size):
    axs_row = int(i / nrows)
    axs_col = int(i % nrows)
    
    action_np = dataset.denormalize_action(action[i].cpu().detach().numpy())
    pred_action_np = dataset.denormalize_action(pred_action[i].cpu().detach().numpy())
    
    if cfg.pos_type == 'corners':
        curr_pos_np = dataset.denormalize_corner(curr_pos[i].cpu().detach().numpy())
        plot_corners(
            ax=axs[axs_col, axs_row],
            curr_pos=curr_pos_np,
            use_img=False,
            img=None,
            plot_action=True,
            actions=(action_np, pred_action_np))
    elif cfg.pos_type == 'rvec_tvec':
        curr_pos_np = dataset.denormalize_pos_rvec_tvec(curr_pos[i].cpu().detach().numpy())
        plot_rvec_tvec(
            ax=axs[axs_col, axs_row],
            curr_pos=curr_pos_np,
            use_img=False,
            img=None,
            plot_action=True,
            actions=(action_np, pred_action_np))
        
plt.savefig(os.path.join(out_dir, 'pil_action_test.jpg'))
    

(4, 8)
